# Files & imports

In [35]:
from pandas import read_csv, DataFrame
import pandas as pd

In [43]:
## https://www.kaggle.com/datasets/mkechinov/direct-messaging/data #

filename_first_purchase = "client_first_purchase_date.csv"
filename_campaigns = "campaigns.csv"

filename_msg = "messages-demo.csv"
file_tag = "ecom_msg"

data_first_purchase = pd.read_csv(filename_first_purchase, na_values="")
data_campaigns = pd.read_csv(filename_campaigns, na_values="")
data_msg = pd.read_csv(filename_msg, na_values="")

# Load File

### >> method 1.1 - random sample with undersample of the negative class (remove 30000 rows) [BEING USED]

In [44]:
import pandas as pd

test = False 

# Sample data based on the 'test_data' flag
def sample_data(data, test_data=False):
    sample_fraction = 0.01 if test_data else 0.0015
    return data.sample(frac=sample_fraction)

# Sample each dataset
data_first_purchase_sampled = sample_data(data_first_purchase, test_data=test)  # Replace test_data=False as needed
data_campaigns_sampled = sample_data(data_campaigns, test_data=test)
data_msg_sampled = sample_data(data_msg, test_data=test)

# Filter the relevant columns
first_purchase_filtered = data_first_purchase_sampled[['client_id', 'first_purchase_date']]
campaign_filtered = data_campaigns_sampled[['id', 'campaign_type', 'channel', 'topic', 'started_at',
                                            'finished_at', 'total_count', 'subject_length',
                                            'subject_with_personalization', 'subject_with_deadline',
                                            'subject_with_emoji', 'subject_with_bonuses',
                                            'subject_with_discount', 'subject_with_saleout']]

# Merge datasets using left join (or inner join if desired)
data_1 = pd.merge(data_msg_sampled, first_purchase_filtered, on='client_id', how='inner')
data = pd.merge(data_1, campaign_filtered, left_on='campaign_id', right_on='id', how='left', suffixes=('_msg', '_cmp'))

# Balancing the final dataset according to the 'is_unsubscribed' column
target_count = data[data['is_unsubscribed'] == "t"].shape[0]
non_target_count = data[data['is_unsubscribed'] == "f"].shape[0]

# Determine how many rows to remove to make the dataset more balanced
rows_to_remove = non_target_count - target_count - 30000
if rows_to_remove > 0:
    data_filtered = data[data['is_unsubscribed'] == "f"].sample(n=non_target_count - rows_to_remove)
    data_final = pd.concat([data[data['is_unsubscribed'] == "t"], data_filtered])
else:
    data_final = data

# Display some rows from the final dataset
print(data_final.head(10))


     id_msg                            message_id  campaign_id   message_type  \
0   4100405  a1132f20-152c-42cf-ad2a-f35f8915599b           27  transactional   
1  18756781  00545786-0fa0-488f-b702-77994f9e658e        18012        trigger   

             client_id channel_msg  category platform email_provider   stream  \
0  1515915625491011914       email       NaN      NaN      gmail.com  desktop   
1  1515915625490599031       email       NaN      NaN      gmail.com  desktop   

   ... started_at finished_at total_count subject_length  \
0  ...        NaN         NaN         NaN            NaN   
1  ...        NaN         NaN         NaN            NaN   

  subject_with_personalization subject_with_deadline subject_with_emoji  \
0                          NaN                   NaN                NaN   
1                          NaN                   NaN                NaN   

  subject_with_bonuses subject_with_discount subject_with_saleout  
0                  NaN               

In [38]:
%run "dslabs_functions.py"

dslabs_functions loaded


# Transformations

## Remove columns

In [45]:
#drop id columns - are not relevant for the prediction of the class

data_final = data_final.drop(columns=['message_id', 'id_msg', 'campaign_id','client_id'])

In [40]:
#drop MV columns - where MV values are around 90% or above. 

#data = data.drop(columns=["category", "blocked_at", "complained_at", "soft_bounced_at", "purchased_at", "hard_bounced_at",
#                          "clicked_first_time_at", "clicked_last_time_at", "unsubscribed_at", "platform"])

## Split 'date' column

In [46]:
import pandas as pd

# Split the 'date'' column to have more information and detail on the:
# - day_of_month
# - day_ok_week 
# - is_weekend
# - moment_of_day
# - hour
# - min
#As the dataset only has data from sepecific month of 2021, we don't need to get the year, quarter.

# Convert the 'date' column to datetime
data_final['date'] = pd.to_datetime(data_final['date'], utc=True)

# Create new columns based on the date column 'event_time'
data_final['week_of_month'] = data_final['date'].apply(lambda x: (x.day - 1) // 7 + 1)
data_final['is_weekend'] = data_final['date'].dt.weekday.apply(lambda x: "weekend" if x >= 5 else "weekday") 
data_final['day_of_week'] = data_final['date'].dt.day_name()  # Monday=0, Sunday=6
data_final['day_of_month'] = data_final['date'].dt.day
data_final['month'] = data_final['date'].dt.month_name()
data_final['hour'] = data_final['date'].dt.hour
data_final['min'] = data_final['date'].dt.minute

# Create function to get time of the day
def categorize_time_of_day(hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'

data_final['time_of_day'] = data_final['hour'].apply(categorize_time_of_day)

# Display the updated DataFrame
data_final.head()


,message_type,channel_msg,category,platform,email_provider,stream,date,sent_at,is_opened,opened_first_time_at,...,subject_with_discount,subject_with_saleout,week_of_month,is_weekend,day_of_week,day_of_month,month,hour,min,time_of_day
0,transactional,email,NaN,NaN,gmail.com,desktop,2021-06-01 00:00:00+00:00,2021-06-01 06:39:45,f,NaN,...,NaN,NaN,1,weekday,Tuesday,1,June,0,0,Night
1,trigger,email,NaN,NaN,gmail.com,desktop,2021-05-15 00:00:00+00:00,2021-05-15 05:39:13,f,NaN,...,NaN,NaN,3,weekend,Saturday,15,May,0,0,Night


# Save df from preprocessing into a csv file

In [42]:
# Definir o caminho para salvar o arquivo CSV
filepath = r'dataset/df_msg_pre_proc.csv'

# Salvar o DataFrame em CSV
data_final.to_csv(filepath, index=False)

data_final.head()


,message_type,channel_msg,category,platform,email_provider,stream,date,sent_at,is_opened,opened_first_time_at,...,subject_with_discount,subject_with_saleout,week_of_month,is_weekend,day_of_week,day_of_month,month,hour,min,time_of_day
0,bulk,mobile_push,NaN,NaN,NaN,desktop,2021-05-12 00:00:00+00:00,2021-05-12 07:38:57,t,2021-05-12 07:39:14,...,NaN,NaN,2,weekday,Wednesday,12,May,0,0,Night
1,bulk,email,NaN,NaN,mail.ru,desktop,2021-05-19 00:00:00+00:00,2021-05-19 07:17:12,f,NaN,...,NaN,NaN,3,weekday,Wednesday,19,May,0,0,Night
2,bulk,mobile_push,NaN,NaN,NaN,desktop,2021-06-11 00:00:00+00:00,2021-06-11 10:00:15,t,2021-06-11 10:00:26,...,NaN,NaN,2,weekday,Friday,11,June,0,0,Night
